# Random Forest training
This is an auto-generated notebook. To reproduce these results, attach this notebook to the **LFICluster-r5d.8xlarge ML** cluster and rerun it.
- Compare trials in the [MLflow experiment](#mlflow/experiments/2652743649175736/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Navigate to the parent notebook [here](#notebook/2652743649175735) (If you launched the AutoML experiment using the Experiments UI, this link isn't very useful.)
- Clone this notebook into your project folder by selecting **File > Clone** in the notebook toolbar.

Runtime Version: _11.1.x-cpu-ml-scala2.12_

In [0]:
import mlflow
import databricks.automl_runtime

target_col = "label"

## Load Data

In [0]:
from mlflow.tracking import MlflowClient
import os
import uuid
import shutil
import pandas as pd

# Create temp directory to download input data from MLflow
input_temp_dir = os.path.join(os.environ["SPARK_LOCAL_DIRS"], "tmp", str(uuid.uuid4())[:8])
os.makedirs(input_temp_dir)


# Download the artifact and read it into a pandas DataFrame
input_client = MlflowClient()
input_data_path = input_client.download_artifacts("d129bc8fc2c54a398dd6dc630fd8a619", "data", input_temp_dir)

df_loaded = pd.read_parquet(os.path.join(input_data_path, "training_data"))
# Delete the temp data
shutil.rmtree(input_temp_dir)

# Preview data
df_loaded.head(5)

,Zip_Codes,married_f_ls_19,married_f_ls_34,married_white,married_black,married_indians,married_asian,ES,ES_ls_10,ES_ls_15,...,high_school,college_level,bachelor_level,graduate_level,house_age_1970_1979,house_age_1960_1969,house_age_1950_1959,house_age_1940_1949,house_age_ls_1939,label
0,60029,11,11,11,11,11,11,11,11,11,...,11,11,11,11,11,11,11,11,11,11
1,60043,78,78,78,78,78,78,78,78,78,...,78,78,78,78,78,78,78,78,78,78
2,60072,42,42,42,42,42,42,42,42,42,...,42,42,42,42,42,42,42,42,42,42
3,60109,27,27,27,27,27,27,27,27,27,...,27,27,27,27,27,27,27,27,27,27
4,60111,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Select supported columns
Select only the columns that are supported. This allows us to train a model that can predict on a dataset that has extra columns that are not used in training.
`[]` are dropped in the pipelines. See the Alerts tab of the AutoML Experiment page for details on why these columns are dropped.

In [0]:
from databricks.automl_runtime.sklearn.column_selector import ColumnSelector
supported_cols = ["house_age_1940_1949", "married_white", "ES_ls_25", "ES_gr_100", "married_indians", "ES", "married_black", "ES_ls_10", "house_age_1970_1979", "house_age_ls_1939", "ES_ls_100", "ES_ls_35", "Zip_Codes", "graduate_level", "ES_ls_65", "married_asian", "married_f_ls_34", "college_level", "ES_ls_50", "house_age_1960_1969", "bachelor_level", "ES_ls_75", "house_age_1950_1959", "ls_high_school", "high_school", "married_f_ls_19", "ES_ls_15"]
col_selector = ColumnSelector(supported_cols)

## Preprocessors

### Numerical columns

Missing values for numerical columns are imputed with mean by default.

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler

num_imputers = []
num_imputers.append(("impute_mean", SimpleImputer(), ["ES", "ES_gr_100", "ES_ls_10", "ES_ls_100", "ES_ls_15", "ES_ls_25", "ES_ls_35", "ES_ls_50", "ES_ls_65", "ES_ls_75", "Zip_Codes", "bachelor_level", "college_level", "graduate_level", "high_school", "house_age_1940_1949", "house_age_1950_1959", "house_age_1960_1969", "house_age_1970_1979", "house_age_ls_1939", "ls_high_school", "married_asian", "married_black", "married_f_ls_19", "married_f_ls_34", "married_indians", "married_white"]))

numerical_pipeline = Pipeline(steps=[
    ("converter", FunctionTransformer(lambda df: df.apply(pd.to_numeric, errors="coerce"))),
    ("imputers", ColumnTransformer(num_imputers)),
    ("standardizer", StandardScaler()),
])

numerical_transformers = [("numerical", numerical_pipeline, ["married_white", "ES_ls_25", "ES_gr_100", "married_indians", "ES", "married_black", "ES_ls_10", "house_age_1970_1979", "house_age_ls_1939", "ES_ls_100", "ES_ls_35", "Zip_Codes", "graduate_level", "ES_ls_65", "married_asian", "ES_ls_15", "married_f_ls_34", "college_level", "ES_ls_50", "house_age_1960_1969", "bachelor_level", "ES_ls_75", "house_age_1950_1959", "ls_high_school", "high_school", "married_f_ls_19", "house_age_1940_1949"])]

In [0]:
from sklearn.compose import ColumnTransformer

transformers = numerical_transformers

preprocessor = ColumnTransformer(transformers, remainder="passthrough", sparse_threshold=0)

## Train - Validation - Test Split
Split the input data into 3 sets:
- Train (60% of the dataset used to train the model)
- Validation (20% of the dataset used to tune the hyperparameters of the model)
- Test (20% of the dataset used to report the true performance of the model on an unseen dataset)

In [0]:
from sklearn.model_selection import train_test_split

split_X = df_loaded.drop([target_col], axis=1)
split_y = df_loaded[target_col]

# Split out train data
X_train, split_X_rem, y_train, split_y_rem = train_test_split(split_X, split_y, train_size=0.6, random_state=30102701, stratify=split_y)

# Split remaining data equally for validation and test
X_val, X_test, y_val, y_test = train_test_split(split_X_rem, split_y_rem, test_size=0.5, random_state=30102701, stratify=split_y_rem)

## Train classification model
- Log relevant metrics to MLflow to track runs
- All the runs are logged under [this MLflow experiment](#mlflow/experiments/2652743649175736/s?orderByKey=metrics.%60val_f1_score%60&orderByAsc=false)
- Change the model parameters and re-run the training cell to log a different trial to the MLflow experiment
- To view the full list of tunable hyperparameters, check the output of the cell below

In [0]:
from sklearn.ensemble import RandomForestClassifier

help(RandomForestClassifier)

Help on class RandomForestClassifier in module sklearn.ensemble._forest:

class RandomForestClassifier(ForestClassifier)
 |  RandomForestClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
 |  
 |  A random forest classifier.
 |  
 |  A random forest is a meta estimator that fits a number of decision tree
 |  classifiers on various sub-samples of the dataset and uses averaging to
 |  improve the predictive accuracy and control over-fitting.
 |  The sub-sample size is controlled with the `max_samples` parameter if
 |  `bootstrap=True` (default), otherwise the whole dataset is used to build
 |  each tree.
 |  
 |  Read more in the :ref:`User Guide <forest>`.
 |  
 |  P

In [0]:
import mlflow
import sklearn
from sklearn import set_config
from sklearn.pipeline import Pipeline

set_config(display="diagram")

skrf_classifier = RandomForestClassifier(
  bootstrap=True,
  criterion="gini",
  max_depth=12,
  max_features=0.8961188850600926,
  min_samples_leaf=0.00932864125414587,
  min_samples_split=0.0011318414866752138,
  n_estimators=1707,
  random_state=30102701,
)

model = Pipeline([
    ("column_selector", col_selector),
    ("preprocessor", preprocessor),
    ("classifier", skrf_classifier),
])

model

Pipeline Pipeline(steps=[('column_selector',
 ColumnSelector(cols=['house_age_1940_1949', 'married_white',
 'ES_ls_25', 'ES_gr_100',
 'married_indians', 'ES', 'married_black',
 'ES_ls_10', 'house_age_1970_1979',
 'house_age_ls_1939', 'ES_ls_100',
 'ES_ls_35', 'Zip_Codes', 'graduate_level',
 'ES_ls_65', 'married_asian',
 'married_f_ls_34', 'college_level',
 'ES_ls_50', 'house_age_1960_1...
 'house_age_1960_1969',
 'bachelor_level', 'ES_ls_75',
 'house_age_1950_1959',
 'ls_high_school',
 'high_school',
 'married_f_ls_19',
 'house_age_1940_1949'])])),
 ('classifier',
 RandomForestClassifier(max_depth=12,
 max_features=0.8961188850600926,
 min_samples_leaf=0.00932864125414587,
 min_samples_split=0.0011318414866752138,
 n_estimators=1707,
 random_state=30102701))]) ColumnSelector ColumnSelector(cols=['house_age_1940_1949', 'married_white', 'ES_ls_25',
 'ES_gr_100', 'married_indians', 'ES', 'married_black',
 'ES_ls_10', 'house_age_1970_1979', 'house_age_ls_1939',
 'ES_ls_100', 'ES_ls_35', 'Zip_Codes', 'graduate_level',
 'ES_ls_65', 'married_asian', 'married_f_ls_34',
 'college_level', 'ES_ls_50', 'house_age_1960_1969',
 'bachelor_level', 'ES_ls_75', 'house_age_1950_1959',
 'ls_high_school', 'high_school', 'married_f_ls_19',
 'ES_ls_15']) preprocessor: ColumnTransformer ColumnTransformer(remainder='passthrough', sparse_threshold=0,
 transformers=[('numerical',
 Pipeline(steps=[('converter',
 FunctionTransformer(func= at 0x7f3524efcca0>)),
 ('imputers',
 ColumnTransformer(transformers=[('impute_mean',
 SimpleImputer(),
 ['ES',
 'ES_gr_100',
 'ES_ls_10',
 'ES_ls_100',
 'ES_ls_15',
 'ES_ls_25',
 'ES_ls_35',
 'ES_ls_50',
 'ES_ls_65',...
 'married_indians', 'ES', 'married_black',
 'ES_ls_10', 'house_age_1970_1979',
 'house_age_ls_1939', 'ES_ls_100', 'ES_ls_35',
 'Zip_Codes', 'graduate_level', 'ES_ls_65',
 'married_asian', 'ES_ls_15',
 'married_f_ls_34', 'college_level',
 'ES_ls_50', 'house_age_1960_1969',
 'bachelor_level', 'ES_ls_75',
 'house_age_1950_1959', 'ls_high_school',
 'high_school', 'married_f_ls_19',
 'house_age_1940_1949'])]) numerical ['married_white', 'ES_ls_25', 'ES_gr_100', 'married_indians', 'ES', 'married_black', 'ES_ls_10', 'house_age_1970_1979', 'house_age_ls_1939', 'ES_ls_100', 'ES_ls_35', 'Zip_Codes', 'graduate_level', 'ES_ls_65', 'married_asian', 'ES_ls_15', 'married_f_ls_34', 'college_level', 'ES_ls_50', 'house_age_1960_1969', 'bachelor_level', 'ES_ls_75', 'house_age_1950_1959', 'ls_high_school', 'high_school', 'married_f_ls_19', 'house_age_1940_1949'] FunctionTransformer FunctionTransformer(func= at 0x7f3524efcca0>) imputers: ColumnTransformer ColumnTransformer(transformers=[('impute_mean', SimpleImputer(),
 ['ES', 'ES_gr_100', 'ES_ls_10', 'ES_ls_100',
 'ES_ls_15', 'ES_ls_25', 'ES_ls_35',
 'ES_ls_50', 'ES_ls_65', 'ES_ls_75',
 'Zip_Codes', 'bachelor_level',
 'college_level', 'graduate_level',
 'high_school', 'house_age_1940_1949',
 'house_age_1950_1959', 'house_age_1960_1969',
 'house_age_1970_1979', 'house_age_ls_1939',
 'ls_high_school', 'married_asian',
 'married_black', 'married_f_ls_19',
 'married_f_ls_34', 'married_indians',
 'married_white'])]) impute_mean ['ES', 'ES_gr_100', 'ES_ls_10', 'ES_ls_100', 'ES_ls_15', 'ES_ls_25', 'ES_ls_35', 'ES_ls_50', 'ES_ls_65', 'ES_ls_75', 'Zip_Codes', 'bachelor_level', 'college_level', 'graduate_level', 'high_school', 'house_age_1940_1949', 'house_age_1950_1959', 'house_age_1960_1969', 'house_age_1970_1979', 'house_age_ls_1939', 'ls_high_school', 'married_asian', 'married_black', 'married_f_ls_19', 'married_f_ls_34', 'married_indians', 'married_white'] SimpleImputer SimpleImputer() StandardScaler StandardScaler() remainder passthrough passthrough RandomForestClassifier RandomForestClassifier(max_depth=12, max_features=0.8961188850600926,
 min_samples_leaf=0.00932864125414587,
 min_samples_split=0.0011318414866752138,
 n_estimators=1707, random_state=30102701)

In [0]:
# Enable automatic logging of input samples, metrics, parameters, and models
mlflow.sklearn.autolog(log_input_examples=True, silent=True)

with mlflow.start_run(experiment_id="2652743649175736", run_name="random_forest") as mlflow_run:
    model.fit(X_train, y_train)
    
    # Training metrics are logged by MLflow autologging
    # Log metrics for the validation set
    skrf_val_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_val, y_val, prefix="val_")

    # Log metrics for the test set
    skrf_test_metrics = mlflow.sklearn.eval_and_log_metrics(model, X_test, y_test, prefix="test_")

    # Display the logged metrics
    skrf_val_metrics = {k.replace("val_", ""): v for k, v in skrf_val_metrics.items()}
    skrf_test_metrics = {k.replace("test_", ""): v for k, v in skrf_test_metrics.items()}
    display(pd.DataFrame([skrf_val_metrics, skrf_test_metrics], index=["validation", "test"]))

/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


/databricks/python/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,precision_score,recall_score,f1_score,accuracy_score,log_loss,roc_auc_score,score
validation,0.896111,0.926667,0.904476,0.926667,0.587121,1.000000,0.926667
test,0.879778,0.920000,0.893444,0.920000,0.588271,0.999788,0.920000


## Feature importance

SHAP is a game-theoretic approach to explain machine learning models, providing a summary plot
of the relationship between features and model output. Features are ranked in descending order of
importance, and impact/color describe the correlation between the feature and the target variable.
- Generating SHAP feature importance is a very memory intensive operation, so to ensure that AutoML can run trials without
  running out of memory, we disable SHAP by default.<br />
  You can set the flag defined below to `shap_enabled = True` and re-run this notebook to see the SHAP plots.
- To reduce the computational overhead of each trial, a single example is sampled from the validation set to explain.<br />
  For more thorough results, increase the sample size of explanations, or provide your own examples to explain.
- SHAP cannot explain models using data with nulls; if your dataset has any, both the background data and
  examples to explain will be imputed using the mode (most frequent values). This affects the computed
  SHAP values, as the imputed samples may not match the actual data distribution.

For more information on how to read Shapley values, see the [SHAP documentation](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html).

In [0]:
# Set this flag to True and re-run the notebook to see the SHAP plots
shap_enabled = False

In [0]:
if shap_enabled:
    from shap import KernelExplainer, summary_plot
    # Sample background data for SHAP Explainer. Increase the sample size to reduce variance.
    train_sample = X_train.sample(n=min(100, X_train.shape[0]), random_state=30102701)

    # Sample some rows from the validation set to explain. Increase the sample size for more thorough results.
    example = X_val.sample(n=min(100, X_val.shape[0]), random_state=30102701)

    # Use Kernel SHAP to explain feature importance on the sampled rows from the validation set.
    predict = lambda x: model.predict(pd.DataFrame(x, columns=X_train.columns))
    explainer = KernelExplainer(predict, train_sample, link="identity")
    shap_values = explainer.shap_values(example, l1_reg=False)
    summary_plot(shap_values, example, class_names=model.classes_)

## Inference
[The MLflow Model Registry](https://docs.databricks.com/applications/mlflow/model-registry.html) is a collaborative hub where teams can share ML models, work together from experimentation to online testing and production, integrate with approval and governance workflows, and monitor ML deployments and their performance. The snippets below show how to add the model trained in this notebook to the model registry and to retrieve it later for inference.

> **NOTE:** The `model_uri` for the model already trained in this notebook can be found in the cell below

### Register to Model Registry
```
model_name = "Example"

model_uri = f"runs:/{ mlflow_run.info.run_id }/model"
registered_model_version = mlflow.register_model(model_uri, model_name)
```

### Load from Model Registry
```
model_name = "Example"
model_version = registered_model_version.version

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")
model.predict(input_X)
```

### Load model without registering
```
model_uri = f"runs:/{ mlflow_run.info.run_id }/model"

model = mlflow.pyfunc.load_model(model_uri)
model.predict(input_X)
```

In [0]:
# model_uri for the generated model
print(f"runs:/{ mlflow_run.info.run_id }/model")

runs:/12668cac9f7a4b808759c7ebec9b0b8f/model
